<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# -*- coding: utf-8 -*-

"""
================================================================================
FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO
================================================================================
Trabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús Lévano Lévano
Universidad: Universidad Oberta de Catalunya (UOC)
Tutor: Miguel Alejandro Ponce Proaño
Fecha: Noviembre 2025

DESCRIPCIÓN:
Fase 2A del análisis comparativo de modelos de segmentación. Calcula métricas
cuantitativas exhaustivas comparando máscaras predichas por 5 modelos contra
ground truth anotado manualmente, con análisis geométrico avanzado usando
Shapely y análisis textural usando Mahotas.

DEPENDENCIAS DE DATOS:
- Requiere Fase 1 completada (índice maestro + configuraciones)
- Utiliza máscaras NPZ de modelos en ubicaciones originales
- Utiliza ground truth desde anotaciones CVAT
- Utiliza características fotográficas del Notebook 00

MODELOS EVALUADOS:
1. Mask2Former (Swin-Large/Tiny, ADE20K/COCO) - 19 configuraciones
2. OneFormer (task-conditioned) - 36 configuraciones
3. SAM 2.0 (con/sin prompts) - 44 configuraciones
4. YOLOv8-seg (nano a xlarge) - 20 configuraciones
5. BodyPix (MobileNetV1) - 8 configuraciones
Total: 127 configuraciones únicas

DATASET:
- 20 fotografías de retrato manualmente anotadas
- Resoluciones variables (típicamente 6016×4016 px)
- Ground truth binario en formato NPZ
- Total evaluaciones: 20 fotos × 127 configs = 2,540 combinaciones

MÉTRICAS CALCULADAS:

A. Métricas Clásicas de Segmentación (5 métricas):
   - IoU (Intersection over Union) - Métrica estándar
   - Dice Coefficient - Solapamiento normalizado
   - Precision - Tasa de verdaderos positivos
   - Recall - Cobertura del ground truth
   - F1-Score - Media armónica precision-recall

B. Métricas Geométricas con Shapely (25 métricas):

   B1. Geometría Básica:
       - Área (pixels)
       - Perímetro (pixels)
       - Bounding box (xmin, ymin, xmax, ymax)
       - Centroide (x, y)
       - Aspect ratio (ancho/alto)
       - Orientación (grados)

   B2. Forma Avanzada:
       - Convex hull (envolvente convexa)
       - Solidity (área / área_convex_hull)
       - Compacidad (4π×área / perímetro²)
       - Circularity
       - Elongation (ratio ejes)
       - Rectangularity (área / área_bbox)

   B3. Contorno:
       - Boundary IoU (solo bordes)
       - Hausdorff Distance (error máximo)
       - Chamfer Distance (error promedio)
       - Smoothness (varianza curvatura)
       - Número de vértices

   B4. Posición Compositiva:
       - Distancia a centro imagen
       - Zona regla de tercios (1-9)
       - Recortes por bordes (%)
       - Espacio negativo

   B5. Fragmentación:
       - Número componentes desconectadas
       - Área componente principal
       - Ratio fragmentación

   B6. Comparación vs Ground Truth:
       - Desplazamiento centroide (Euclidean)
       - Diferencia orientación (grados)
       - Ratio solidity (pred/GT)
       - Diferencia compacidad
       - Symmetric difference area

C. Métricas Texturales con Mahotas (30 métricas):

   C1. Texturas Haralick - Región Interior (13 features):
       - Angular Second Moment
       - Contrast
       - Correlation
       - Sum of Squares Variance
       - Inverse Difference Moment
       - Sum Average
       - Sum Variance
       - Sum Entropy
       - Entropy
       - Difference Variance
       - Difference Entropy
       - Information Measure Correlation 1
       - Information Measure Correlation 2

   C2. Texturas Haralick - Región Borde (13 features):
       - Mismas features calculadas en borde de 5px
       - Evalúa calidad de separación persona-fondo

   C3. Estadísticas Básicas (4 métricas):
       - Media, std, min, max de intensidad

TOTAL MÉTRICAS POR CONFIGURACIÓN: ~60 métricas

PROCESAMIENTO:
- Incremental con sistema de checkpoint resumible
- Guardado automático después de cada fotografía procesada
- Gestión optimizada de memoria para Google Colab free tier
- Manejo robusto de configuraciones faltantes
- Logging detallado para trazabilidad

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2_evaluacion/
├── metricas_individuales/
│   ├── _DSC0036_metricas_completas.json
│   ├── _DSC0592_metricas_completas.json
│   └── ... (20 archivos JSON)
│
├── metricas_agregadas/
│   ├── todas_metricas.csv                  # Tabla plana: 2540 filas × ~65 columnas
│   ├── ranking_configuraciones.csv         # TOP-10 configs ordenadas por IoU
│   ├── estadisticas_por_modelo.json        # Media, std, min, max por modelo
│   ├── configuraciones_omitidas.json       # Registro de omisiones
│   └── datos_visualizaciones.json          # Datos pre-formateados para plots
│
├── checkpoint_fase2a.json                  # Estado de procesamiento
└── INFORME_FASE2A.md                       # Documento narrativo académico

REFERENCIAS METODOLÓGICAS:
- Lin, T.Y., et al. (2014). Microsoft COCO: Common Objects in Context
- Kirillov, A., et al. (2023). Segment Anything
- Shapely Documentation (2024). Geometric Operations
- Haralick, R.M., et al. (1973). Textural Features for Image Classification
- Gonzalez, R.C. & Woods, R.E. (2018). Digital Image Processing (4th ed.)

COMPATIBILIDAD:
- Google Colab Free Tier
- Python 3.10+
- Requiere: numpy, pandas, opencv-python, shapely, mahotas, Pillow, scikit-image
================================================================================
"""

'\n================================================================================\nFASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO\n================================================================================\nTrabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación\nen Fotografía de Retrato\n\nAutor: Jesús Lévano Lévano\nUniversidad: Universidad Oberta de Catalunya (UOC)\nTutor: Miguel Alejandro Ponce Proaño\nFecha: Noviembre 2025\n\nDESCRIPCIÓN:\nFase 2A del análisis comparativo de modelos de segmentación. Calcula métricas\ncuantitativas exhaustivas comparando máscaras predichas por 5 modelos contra\nground truth anotado manualmente, con análisis geométrico avanzado usando\nShapely y análisis textural usando Mahotas.\n\nDEPENDENCIAS DE DATOS:\n- Requiere Fase 1 completada (índice maestro + configuraciones)\n- Utiliza máscaras NPZ de modelos en ubicaciones originales\n- Utiliza ground truth desde anotaciones CVAT\n- Utiliza características fotográficas d

In [28]:
!pip install -q shapely mahotas scikit-image

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# =============================================================================
# IMPORTACIÓN DE LIBRERÍAS
# =============================================================================

# Librerías estándar de Python
import json
import logging
import sys
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any, Set

# Librerías de procesamiento numérico y datos
import numpy as np
import pandas as pd

# Librerías de procesamiento de imágenes
import cv2
import mahotas
from PIL import Image
from scipy.ndimage import distance_transform_edt
from scipy.spatial.distance import directed_hausdorff
from skimage import measure

# Shapely para análisis geométrico
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely.validation import make_valid

In [31]:
# =============================================================================
# DEFINICIÓN DE ESTRUCTURAS DE DATOS
# =============================================================================

class NivelLogging(Enum):
    """
    Enumeración de niveles de logging para control de verbosidad.

    Niveles:
    - MINIMAL: Solo resumen por fotografía
    - NORMAL: Progreso con detalle moderado (recomendado)
    - DETAILED: Información detallada por configuración
    - DEBUG: Máximo detalle para debugging
    """
    MINIMAL = "minimal"
    NORMAL = "normal"
    DETAILED = "detailed"
    DEBUG = "debug"

In [32]:
@dataclass
class ConfiguracionFase2A:
    """
    Dataclass para configuración global de Fase 2A.

    Attributes:
        ruta_base_tfm: Directorio raíz del proyecto TFM
        max_dimension_mahotas: Redimensión para Mahotas (optimización)
        ancho_borde_px: Ancho del borde para análisis textural
        usar_checkpoint: Habilita sistema de checkpoint resumible
        guardar_json_individuales: Guardar JSON por fotografía
        guardar_csv_consolidado: Guardar CSV con todas las métricas
        guardar_estadisticas: Calcular estadísticas por modelo
        nivel_logging: Nivel de verbosidad del logging
    """
    ruta_base_tfm: Path
    max_dimension_mahotas: int = 1024
    ancho_borde_px: int = 5
    usar_checkpoint: bool = True
    guardar_json_individuales: bool = True
    guardar_csv_consolidado: bool = True
    guardar_estadisticas: bool = True
    nivel_logging: NivelLogging = NivelLogging.NORMAL

    @property
    def ruta_fase1(self) -> Path:
        """Ruta al directorio de resultados de Fase 1."""
        return self.ruta_base_tfm / "3_Analisis" / "fase1_integracion"

    @property
    def ruta_salida(self) -> Path:
        """Ruta al directorio de salida de Fase 2A."""
        return self.ruta_base_tfm / "3_Analisis" / "fase2_evaluacion"

    @property
    def ruta_ground_truth(self) -> Path:
        """Ruta al directorio de ground truth."""
        return self.ruta_base_tfm / "0_Imagenes_CVAT" / "ground_truth_masks"

    @property
    def ruta_caracteristicas(self) -> Path:
        """Ruta al directorio de características fotográficas."""
        return self.ruta_base_tfm / "1_Caracteristicas" / "json"

    def crear_directorios(self) -> None:
        """Crea la estructura de directorios de salida."""
        (self.ruta_salida / "metricas_individuales").mkdir(parents=True, exist_ok=True)
        (self.ruta_salida / "metricas_agregadas").mkdir(parents=True, exist_ok=True)


@dataclass
class CheckpointFase2A:
    """
    Dataclass para gestión de checkpoint de procesamiento.

    Permite reanudar el procesamiento desde el punto de interrupción,
    evitando reprocesar fotografías ya completadas.

    Attributes:
        fotos_procesadas: Conjunto de códigos de fotografías completadas
        fotos_pendientes: Lista de códigos de fotografías por procesar
        configuraciones_totales: Número total de configuraciones
        ultima_actualizacion: Timestamp de última actualización
    """
    fotos_procesadas: Set[str] = field(default_factory=set)
    fotos_pendientes: List[str] = field(default_factory=list)
    configuraciones_totales: int = 0
    ultima_actualizacion: Optional[str] = None

    def to_dict(self) -> Dict[str, Any]:
        """Serializa checkpoint a diccionario JSON-compatible."""
        return {
            "fotos_procesadas": list(self.fotos_procesadas),
            "fotos_pendientes": self.fotos_pendientes,
            "configuraciones_totales": self.configuraciones_totales,
            "ultima_actualizacion": self.ultima_actualizacion
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'CheckpointFase2A':
        """Reconstruye checkpoint desde diccionario."""
        return cls(
            fotos_procesadas=set(data.get("fotos_procesadas", [])),
            fotos_pendientes=data.get("fotos_pendientes", []),
            configuraciones_totales=data.get("configuraciones_totales", 0),
            ultima_actualizacion=data.get("ultima_actualizacion")
        )

In [43]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def configurar_logging(nivel: NivelLogging, nombre_modulo: str = "Fase2A") -> logging.Logger:
    """
    Configuración del sistema de logging con formato académico.

    Args:
        nivel: Nivel de logging deseado
        nombre_modulo: Nombre del módulo para identificación

    Returns:
        Logger configurado
    """
    logger = logging.getLogger(nombre_modulo)
    logger.setLevel(logging.DEBUG)

    if logger.handlers:
        logger.handlers.clear()

    handler = logging.StreamHandler(sys.stdout)

    if nivel == NivelLogging.MINIMAL:
        handler.setLevel(logging.WARNING)
    elif nivel == NivelLogging.NORMAL:
        handler.setLevel(logging.INFO)
    elif nivel == NivelLogging.DETAILED:
        handler.setLevel(logging.INFO)
    else:
        handler.setLevel(logging.DEBUG)

    formatter = logging.Formatter(
        fmt='[%(asctime)s] %(levelname)-8s | %(message)s',
        datefmt='%H:%M:%S'
    )

    handler.setFormatter(formatter)
    logger.addHandler(handler)

    return logger


def guardar_json_seguro(datos: Any, ruta_archivo: Path, logger: logging.Logger) -> bool:
    """
    Guarda datos en formato JSON con manejo robusto de errores.

    Incluye conversión automática de tipos NumPy y sets a formatos JSON-compatibles.

    Args:
        datos: Datos a serializar
        ruta_archivo: Path donde guardar el JSON
        logger: Logger para registrar operaciones

    Returns:
        True si guardado exitoso, False en caso contrario
    """
    try:
        class NumpyEncoder(json.JSONEncoder):
            """Encoder personalizado para tipos NumPy y estructuras complejas."""
            def default(self, obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                if isinstance(obj, np.floating):
                    return float(obj)
                if isinstance(obj, np.ndarray):
                    return obj.tolist()
                if isinstance(obj, set):
                    return list(obj)
                if isinstance(obj, Path):
                    return str(obj)
                return super().default(obj)

        ruta_archivo.parent.mkdir(parents=True, exist_ok=True)

        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False, cls=NumpyEncoder)

        return True

    except Exception as e:
        logger.error(f"Error guardando JSON en {ruta_archivo}: {e}")
        return False


def cargar_mascara_npz(ruta_npz: Path, umbral: Optional[float] = None, logger: Optional[logging.Logger] = None) -> Optional[np.ndarray]:
    """
    Carga máscara desde archivo NPZ con manejo de múltiples estructuras.

    Soporta tres estructuras:
    1. Unificada: clave 'person_mask' (SAM, Mask2Former, OneFormer, YOLO)
    2. Por umbral: claves 'mask_X.XX' (BodyPix)
    3. Ground Truth CVAT: clave 'masks' con array 3D [1, H, W]

    Args:
        ruta_npz: Path al archivo NPZ
        umbral: Umbral específico para modelos por-umbral (ej: 0.50)
        logger: Logger para debugging

    Returns:
        Array booleano 2D con la máscara, o None si error

    Raises:
        FileNotFoundError: Si el archivo no existe
        ValueError: Si estructura NPZ inválida o clave faltante
    """
    if not ruta_npz.exists():
        if logger:
            logger.warning(f"Archivo NPZ no encontrado: {ruta_npz}")
        return None

    try:
        with np.load(ruta_npz, allow_pickle=False) as data:
            claves_disponibles = list(data.keys())

            if logger:
                logger.debug(f"Claves en NPZ: {claves_disponibles}")

            # Caso 1: Estructura unificada (modelos)
            if 'person_mask' in claves_disponibles:
                mascara = data['person_mask']

                if logger:
                    logger.debug(f"Cargada máscara unificada, shape: {mascara.shape}")

            # Caso 2: Ground Truth CVAT (estructura con 'masks')
            elif 'masks' in claves_disponibles:
                mascara = data['masks']

                if logger:
                    logger.debug(f"Cargada máscara Ground Truth CVAT, shape: {mascara.shape}")

                # El ground truth de CVAT suele venir como [1, H, W] o [H, W]
                if mascara.ndim == 3:
                    # Tomar el primer canal
                    mascara = mascara[0]
                    if logger:
                        logger.debug(f"Reducida dimensión: {mascara.shape}")

            # Caso 3: Estructura por-umbral (BodyPix)
            else:
                claves_mascara = [k for k in claves_disponibles if k.startswith('mask_')]

                if not claves_mascara:
                    if logger:
                        logger.error(f"No se encontró 'person_mask', 'masks' ni 'mask_X.XX' en {ruta_npz}")
                    return None

                if umbral is not None:
                    clave_objetivo = f"mask_{umbral:.2f}"

                    if clave_objetivo not in claves_disponibles:
                        if logger:
                            logger.warning(f"Umbral {umbral:.2f} no encontrado. Disponibles: {claves_mascara}")
                        return None

                    mascara = data[clave_objetivo]

                    if logger:
                        logger.debug(f"Cargada máscara por umbral {clave_objetivo}, shape: {mascara.shape}")

                else:
                    if logger:
                        logger.error(f"Estructura por-umbral requiere especificar 'umbral'. Disponibles: {claves_mascara}")
                    return None

            # Validación de dimensiones
            if mascara.ndim != 2:
                if logger:
                    logger.error(f"Máscara con dimensiones inválidas: {mascara.shape}")
                return None

            mascara_bool = mascara.astype(bool)

            return mascara_bool

    except Exception as e:
        if logger:
            logger.error(f"Error cargando NPZ {ruta_npz}: {e}")
        return None


def mascara_a_shapely(mascara: np.ndarray, logger: Optional[logging.Logger] = None) -> Optional[Polygon | MultiPolygon]:
    """
    Convierte máscara binaria a polígonos Shapely.

    Utiliza find_contours de scikit-image para detectar contornos y los
    convierte en geometrías Shapely válidas. Maneja múltiples componentes
    desconectadas y aplica validación geométrica.

    Args:
        mascara: Array 2D booleano con la máscara
        logger: Logger opcional para debugging

    Returns:
        Polygon o MultiPolygon válido, o None si error

    Notes:
        - Invierte coordenadas Y debido a convención imagen vs Shapely
        - Filtra polígonos con menos de 3 vértices
        - Aplica make_valid() para corregir auto-intersecciones
        - Une polígonos múltiples usando unary_union para evitar errores
    """
    try:
        # Asegurar que la máscara es binaria
        mascara_binaria = mascara.astype(np.uint8)

        # Encontrar contornos
        contornos = measure.find_contours(mascara_binaria, level=0.5)

        if len(contornos) == 0:
            if logger:
                logger.debug("No se encontraron contornos en la máscara")
            return None

        poligonos_validos = []

        for contorno in contornos:
            # Invertir coordenadas Y (convención imagen vs Shapely)
            coordenadas = [(x, mascara.shape[0] - y) for y, x in contorno]

            # Filtrar contornos con muy pocos puntos
            if len(coordenadas) < 3:
                continue

            try:
                # Crear polígono
                poligono = Polygon(coordenadas)

                # Validar y corregir si es necesario
                if not poligono.is_valid:
                    poligono = make_valid(poligono)

                # Verificar que sigue siendo válido después de make_valid
                if poligono.is_valid and not poligono.is_empty:
                    # Si make_valid devolvió un MultiPolygon, extraer polígonos individuales
                    if isinstance(poligono, MultiPolygon):
                        for poly in poligono.geoms:
                            if poly.is_valid and not poly.is_empty:
                                poligonos_validos.append(poly)
                    else:
                        poligonos_validos.append(poligono)

            except Exception as e_poly:
                if logger:
                    logger.debug(f"Error creando polígono individual: {e_poly}")
                continue

        if len(poligonos_validos) == 0:
            if logger:
                logger.debug("No se generaron polígonos válidos")
            return None

        # Si hay un solo polígono, retornarlo directamente
        if len(poligonos_validos) == 1:
            return poligonos_validos[0]

        # Si hay múltiples polígonos, usar unary_union para combinarlos correctamente
        try:
            geometria_unida = unary_union(poligonos_validos)

            # Validar el resultado final
            if not geometria_unida.is_valid:
                geometria_unida = make_valid(geometria_unida)

            if geometria_unida.is_empty:
                if logger:
                    logger.warning("La unión de polígonos resultó vacía")
                return None

            return geometria_unida

        except Exception as e_union:
            if logger:
                logger.error(f"Error en unary_union: {e_union}")

            # Fallback: retornar MultiPolygon directamente
            try:
                return MultiPolygon(poligonos_validos)
            except Exception as e_mp:
                if logger:
                    logger.error(f"Error creando MultiPolygon: {e_mp}")
                return None

    except Exception as e:
        if logger:
            logger.error(f"Error convirtiendo máscara a Shapely: {e}")
        return None

In [34]:
# =============================================================================
# CALCULADORAS DE MÉTRICAS
# =============================================================================

def calcular_metricas_clasicas(mascara_pred: np.ndarray, mascara_gt: np.ndarray) -> Dict[str, float]:
    """
    Calcula métricas clásicas de segmentación.

    Métricas calculadas:
    - IoU (Intersection over Union): |A ∩ B| / |A ∪ B|
    - Dice Coefficient: 2|A ∩ B| / (|A| + |B|)
    - Precision: TP / (TP + FP)
    - Recall: TP / (TP + FN)
    - F1-Score: 2 × (Precision × Recall) / (Precision + Recall)

    Args:
        mascara_pred: Máscara predicha (booleano 2D)
        mascara_gt: Ground truth (booleano 2D)

    Returns:
        Diccionario con las 5 métricas, o dict con NaN si error

    References:
        Lin et al. (2014). Microsoft COCO: Common Objects in Context
    """
    try:
        interseccion = np.logical_and(mascara_pred, mascara_gt).sum()
        union = np.logical_or(mascara_pred, mascara_gt).sum()

        tp = interseccion
        fp = mascara_pred.sum() - interseccion
        fn = mascara_gt.sum() - interseccion

        iou = interseccion / union if union > 0 else 0.0

        suma_areas = mascara_pred.sum() + mascara_gt.sum()
        dice = (2 * interseccion) / suma_areas if suma_areas > 0 else 0.0

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        return {
            "iou": float(iou),
            "dice": float(dice),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1)
        }

    except Exception:
        return {
            "iou": np.nan,
            "dice": np.nan,
            "precision": np.nan,
            "recall": np.nan,
            "f1_score": np.nan
        }


def calcular_metricas_shapely(geometria: Polygon | MultiPolygon, dimensiones_imagen: Tuple[int, int], logger: Optional[logging.Logger] = None) -> Dict[str, float]:
    """
    Calcula métricas geométricas usando Shapely.

    Métricas de forma (13):
    - área, perímetro, centroide, bounding box
    - aspect_ratio, orientación
    - solidity, compacidad, circularity
    - elongation, rectangularity
    - num_componentes, ratio_componente_principal

    Métricas compositivas (4):
    - distancia_centro, zona_tercios
    - recorte_bordes, espacio_negativo

    Args:
        geometria: Polygon o MultiPolygon de Shapely
        dimensiones_imagen: Tupla (altura, ancho) de la imagen
        logger: Logger opcional

    Returns:
        Diccionario con ~17 métricas geométricas

    Notes:
        - Solidity: compacidad considerando concavidades
        - Compacidad: círculo perfecto = 1.0
        - Zona tercios: 1-9 según regla fotográfica de tercios
    """
    try:
        altura_img, ancho_img = dimensiones_imagen

        # Métricas básicas
        area = geometria.area
        perimetro = geometria.length
        centroide = geometria.centroid
        bbox = geometria.bounds  # (minx, miny, maxx, maxy)

        ancho_bbox = bbox[2] - bbox[0]
        alto_bbox = bbox[3] - bbox[1]
        aspect_ratio = ancho_bbox / alto_bbox if alto_bbox > 0 else 0.0

        # Orientación
        try:
            coords = np.array(geometria.exterior.coords if isinstance(geometria, Polygon) else geometria.geoms[0].exterior.coords)
            orientacion = np.arctan2(coords[-1, 1] - coords[0, 1], coords[-1, 0] - coords[0, 0]) * 180 / np.pi
        except:
            orientacion = 0.0

        # Solidity
        convex_hull = geometria.convex_hull
        solidity = area / convex_hull.area if convex_hull.area > 0 else 0.0

        # Compacidad
        compacidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0.0

        circularity = compacidad

        # Elongation
        elongation = max(ancho_bbox, alto_bbox) / min(ancho_bbox, alto_bbox) if min(ancho_bbox, alto_bbox) > 0 else 1.0

        # Rectangularity
        area_bbox = ancho_bbox * alto_bbox
        rectangularity = area / area_bbox if area_bbox > 0 else 0.0

        # Fragmentación
        if isinstance(geometria, MultiPolygon):
            num_componentes = len(geometria.geoms)
            areas_componentes = [g.area for g in geometria.geoms]
            area_principal = max(areas_componentes)
            ratio_componente_principal = area_principal / area if area > 0 else 1.0
        else:
            num_componentes = 1
            area_principal = area
            ratio_componente_principal = 1.0

        # Posición compositiva
        centro_imagen = Point(ancho_img / 2, altura_img / 2)
        distancia_centro = centroide.distance(centro_imagen)

        # Zona de tercios
        tercio_ancho = ancho_img / 3
        tercio_alto = altura_img / 3

        col_tercio = int(centroide.x // tercio_ancho)
        fila_tercio = int(centroide.y // tercio_alto)

        col_tercio = min(col_tercio, 2)
        fila_tercio = min(fila_tercio, 2)

        zona_tercios = fila_tercio * 3 + col_tercio + 1

        # Recorte por bordes
        margen = 10
        recorte_izq = max(0, margen - bbox[0])
        recorte_der = max(0, bbox[2] - (ancho_img - margen))
        recorte_sup = max(0, margen - bbox[1])
        recorte_inf = max(0, bbox[3] - (altura_img - margen))

        recorte_total = recorte_izq + recorte_der + recorte_sup + recorte_inf
        perimetro_bbox = 2 * (ancho_bbox + alto_bbox)
        porcentaje_recorte = recorte_total / perimetro_bbox if perimetro_bbox > 0 else 0.0

        # Espacio negativo
        area_imagen_total = altura_img * ancho_img
        espacio_negativo = 1.0 - (area / area_imagen_total)

        return {
            "area_px": float(area),
            "perimetro_px": float(perimetro),
            "centroide_x": float(centroide.x),
            "centroide_y": float(centroide.y),
            "bbox_xmin": float(bbox[0]),
            "bbox_ymin": float(bbox[1]),
            "bbox_xmax": float(bbox[2]),
            "bbox_ymax": float(bbox[3]),
            "aspect_ratio": float(aspect_ratio),
            "orientacion_grados": float(orientacion),
            "solidity": float(solidity),
            "compacidad": float(compacidad),
            "circularity": float(circularity),
            "elongation": float(elongation),
            "rectangularity": float(rectangularity),
            "num_componentes": int(num_componentes),
            "ratio_componente_principal": float(ratio_componente_principal),
            "distancia_centro_px": float(distancia_centro),
            "zona_tercios": int(zona_tercios),
            "recorte_bordes_porcentaje": float(porcentaje_recorte),
            "espacio_negativo": float(espacio_negativo)
        }

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas Shapely: {e}")

        return {k: np.nan for k in [
            "area_px", "perimetro_px", "centroide_x", "centroide_y",
            "bbox_xmin", "bbox_ymin", "bbox_xmax", "bbox_ymax",
            "aspect_ratio", "orientacion_grados", "solidity", "compacidad",
            "circularity", "elongation", "rectangularity", "num_componentes",
            "ratio_componente_principal", "distancia_centro_px", "zona_tercios",
            "recorte_bordes_porcentaje", "espacio_negativo"
        ]}


def calcular_metricas_comparativas(geom_pred: Polygon | MultiPolygon, geom_gt: Polygon | MultiPolygon, mascara_pred: np.ndarray, mascara_gt: np.ndarray, logger: Optional[logging.Logger] = None) -> Dict[str, float]:
    """
    Calcula métricas de comparación entre predicción y ground truth.

    Métricas de contorno (3):
    - boundary_iou: IoU calculado solo sobre bordes
    - hausdorff_distance: Máxima desviación entre contornos
    - chamfer_distance: Desviación promedio

    Métricas de forma (4):
    - desplazamiento_centroide: Distancia Euclidiana entre centroides
    - diferencia_orientacion: Diferencia angular en grados
    - ratio_solidity: pred_solidity / gt_solidity
    - diferencia_compacidad: Diferencia absoluta

    Métricas de área (1):
    - symmetric_difference_area: Área de diferencia simétrica

    Args:
        geom_pred: Geometría predicha
        geom_gt: Geometría ground truth
        mascara_pred: Máscara predicha (para boundary IoU)
        mascara_gt: Máscara GT (para boundary IoU)
        logger: Logger opcional

    Returns:
        Diccionario con 8 métricas comparativas

    Notes:
        - Boundary IoU: Evalúa específicamente calidad de bordes
        - Hausdorff: Detecta outliers en la segmentación
        - Chamfer: Métrica más robusta a outliers que Hausdorff
    """
    try:
        # Boundary IoU
        boundary_pred = cv2.Canny(mascara_pred.astype(np.uint8) * 255, 100, 200) > 0
        boundary_gt = cv2.Canny(mascara_gt.astype(np.uint8) * 255, 100, 200) > 0

        boundary_interseccion = np.logical_and(boundary_pred, boundary_gt).sum()
        boundary_union = np.logical_or(boundary_pred, boundary_gt).sum()

        boundary_iou = boundary_interseccion / boundary_union if boundary_union > 0 else 0.0

        # Hausdorff y Chamfer Distance
        coords_pred = np.column_stack(np.where(boundary_pred))
        coords_gt = np.column_stack(np.where(boundary_gt))

        if len(coords_pred) > 0 and len(coords_gt) > 0:
            hausdorff_dist = max(
                directed_hausdorff(coords_pred, coords_gt)[0],
                directed_hausdorff(coords_gt, coords_pred)[0]
            )

            dist_pred_to_gt = distance_transform_edt(~boundary_gt)[boundary_pred]
            dist_gt_to_pred = distance_transform_edt(~boundary_pred)[boundary_gt]

            chamfer_dist = (dist_pred_to_gt.sum() + dist_gt_to_pred.sum()) / (len(dist_pred_to_gt) + len(dist_gt_to_pred))
        else:
            hausdorff_dist = np.nan
            chamfer_dist = np.nan

        # Desplazamiento centroide
        centroide_pred = geom_pred.centroid
        centroide_gt = geom_gt.centroid
        desplazamiento_centroide = centroide_pred.distance(centroide_gt)

        # Diferencia orientación
        try:
            coords_pred_poly = np.array(geom_pred.exterior.coords if isinstance(geom_pred, Polygon) else geom_pred.geoms[0].exterior.coords)
            coords_gt_poly = np.array(geom_gt.exterior.coords if isinstance(geom_gt, Polygon) else geom_gt.geoms[0].exterior.coords)

            orient_pred = np.arctan2(coords_pred_poly[-1, 1] - coords_pred_poly[0, 1], coords_pred_poly[-1, 0] - coords_pred_poly[0, 0]) * 180 / np.pi
            orient_gt = np.arctan2(coords_gt_poly[-1, 1] - coords_gt_poly[0, 1], coords_gt_poly[-1, 0] - coords_gt_poly[0, 0]) * 180 / np.pi

            diferencia_orientacion = abs(orient_pred - orient_gt)
        except:
            diferencia_orientacion = np.nan

        # Ratio solidity
        solidity_pred = geom_pred.area / geom_pred.convex_hull.area if geom_pred.convex_hull.area > 0 else 0.0
        solidity_gt = geom_gt.area / geom_gt.convex_hull.area if geom_gt.convex_hull.area > 0 else 0.0

        ratio_solidity = solidity_pred / solidity_gt if solidity_gt > 0 else np.nan

        # Diferencia compacidad
        perimetro_pred = geom_pred.length
        perimetro_gt = geom_gt.length

        compacidad_pred = (4 * np.pi * geom_pred.area) / (perimetro_pred ** 2) if perimetro_pred > 0 else 0.0
        compacidad_gt = (4 * np.pi * geom_gt.area) / (perimetro_gt ** 2) if perimetro_gt > 0 else 0.0

        diferencia_compacidad = abs(compacidad_pred - compacidad_gt)

        # Symmetric difference
        sym_diff = geom_pred.symmetric_difference(geom_gt)
        symmetric_difference_area = sym_diff.area

        return {
            "boundary_iou": float(boundary_iou),
            "hausdorff_distance": float(hausdorff_dist),
            "chamfer_distance": float(chamfer_dist),
            "desplazamiento_centroide_px": float(desplazamiento_centroide),
            "diferencia_orientacion_grados": float(diferencia_orientacion),
            "ratio_solidity": float(ratio_solidity),
            "diferencia_compacidad": float(diferencia_compacidad),
            "symmetric_difference_area_px": float(symmetric_difference_area)
        }

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas comparativas: {e}")

        return {k: np.nan for k in [
            "boundary_iou", "hausdorff_distance", "chamfer_distance",
            "desplazamiento_centroide_px", "diferencia_orientacion_grados",
            "ratio_solidity", "diferencia_compacidad", "symmetric_difference_area_px"
        ]}


def calcular_metricas_mahotas(imagen_rgb: np.ndarray, mascara: np.ndarray, ancho_borde: int = 5, max_dimension: int = 1024, logger: Optional[logging.Logger] = None) -> Dict[str, float]:
    """
    Calcula métricas texturales usando Mahotas.

    Extrae 3 tipos de métricas:
    1. Texturas Haralick interior (13 features)
    2. Texturas Haralick borde (13 features)
    3. Estadísticas básicas (4 valores)

    Args:
        imagen_rgb: Imagen original en RGB
        mascara: Máscara binaria de la región
        ancho_borde: Ancho del borde en pixels
        max_dimension: Redimensión máxima (optimización)
        logger: Logger opcional

    Returns:
        Diccionario con 30 métricas texturales

    Notes:
        - Haralick: Calcula co-ocurrencia de grises
        - Borde: Evalúa calidad de transición persona-fondo
        - Redimensión: Trade-off velocidad vs precisión

    References:
        Haralick et al. (1973). Textural Features for Image Classification
    """
    try:
        altura, ancho = imagen_rgb.shape[:2]
        factor_escala = 1.0

        # Redimensionar si excede límite
        if max(altura, ancho) > max_dimension:
            factor_escala = max_dimension / max(altura, ancho)
            nuevo_ancho = int(ancho * factor_escala)
            nuevo_alto = int(altura * factor_escala)

            imagen_rgb = cv2.resize(imagen_rgb, (nuevo_ancho, nuevo_alto), interpolation=cv2.INTER_AREA)
            mascara = cv2.resize(mascara.astype(np.uint8), (nuevo_ancho, nuevo_alto), interpolation=cv2.INTER_NEAREST).astype(bool)

        # Convertir a escala de grises
        imagen_gris = cv2.cvtColor(imagen_rgb, cv2.COLOR_RGB2GRAY)

        # Región interior
        region_persona = imagen_gris * mascara

        if region_persona[mascara].size == 0:
            raise ValueError("Región de persona vacía")

        haralick_interior = mahotas.features.haralick(region_persona, ignore_zeros=True).mean(axis=0)

        # Región borde
        kernel = np.ones((ancho_borde * 2 + 1, ancho_borde * 2 + 1), np.uint8)
        mascara_dilatada = cv2.dilate(mascara.astype(np.uint8), kernel, iterations=1).astype(bool)
        mascara_erosionada = cv2.erode(mascara.astype(np.uint8), kernel, iterations=1).astype(bool)

        mascara_borde = np.logical_and(mascara_dilatada, ~mascara_erosionada)

        region_borde = imagen_gris * mascara_borde

        if region_borde[mascara_borde].size == 0:
            haralick_borde = np.full(13, np.nan)
        else:
            haralick_borde = mahotas.features.haralick(region_borde, ignore_zeros=True).mean(axis=0)

        # Estadísticas básicas
        intensidades = region_persona[mascara]

        media_intensidad = float(np.mean(intensidades))
        std_intensidad = float(np.std(intensidades))
        min_intensidad = float(np.min(intensidades))
        max_intensidad = float(np.max(intensidades))

        # Construir diccionario de resultados
        nombres_haralick = [
            "angular_second_moment", "contrast", "correlation",
            "sum_of_squares_variance", "inverse_difference_moment",
            "sum_average", "sum_variance", "sum_entropy",
            "entropy", "difference_variance", "difference_entropy",
            "information_measure_correlation_1", "information_measure_correlation_2"
        ]

        resultado = {}

        for i, nombre in enumerate(nombres_haralick):
            resultado[f"haralick_interior_{nombre}"] = float(haralick_interior[i])
            resultado[f"haralick_borde_{nombre}"] = float(haralick_borde[i])

        resultado.update({
            "intensidad_media": media_intensidad,
            "intensidad_std": std_intensidad,
            "intensidad_min": min_intensidad,
            "intensidad_max": max_intensidad
        })

        return resultado

    except Exception as e:
        if logger:
            logger.error(f"Error calculando métricas Mahotas: {e}")

        nombres_haralick = [
            "angular_second_moment", "contrast", "correlation",
            "sum_of_squares_variance", "inverse_difference_moment",
            "sum_average", "sum_variance", "sum_entropy",
            "entropy", "difference_variance", "difference_entropy",
            "information_measure_correlation_1", "information_measure_correlation_2"
        ]

        resultado = {}
        for nombre in nombres_haralick:
            resultado[f"haralick_interior_{nombre}"] = np.nan
            resultado[f"haralick_borde_{nombre}"] = np.nan

        resultado.update({
            "intensidad_media": np.nan,
            "intensidad_std": np.nan,
            "intensidad_min": np.nan,
            "intensidad_max": np.nan
        })

        return resultado

In [35]:
# =============================================================================
# SISTEMA DE SELECCIÓN INTERACTIVA
# =============================================================================

class SelectorModelos:
    """
    Sistema de selección interactiva de familias de modelos a evaluar.
    """

    def __init__(self, indice_maestro: Dict, logger: logging.Logger):
        """
        Inicializa el selector.

        Args:
            indice_maestro: Índice maestro consolidado de Fase 1
            logger: Sistema de logging
        """
        self.indice = indice_maestro
        self.logger = logger
        self.modelos_por_familia = self._extraer_familias()

    def _extraer_familias(self) -> Dict[str, List[str]]:
        """
        Extrae configuraciones agrupadas por familia desde índice maestro.

        Returns:
            Diccionario {familia: [lista_de_codigos_config]}
        """
        familias = {}

        # Iterar sobre todas las fotos
        for foto_id, datos_foto in self.indice.items():
            modelos_disponibles = datos_foto.get('modelos_disponibles', {})

            # Iterar sobre todas las configuraciones de esta foto
            for codigo_config in modelos_disponibles.keys():
                # Extraer familia del código
                familia = self._extraer_familia_de_codigo(codigo_config)

                if familia not in familias:
                    familias[familia] = set()

                familias[familia].add(codigo_config)

        # Convertir sets a listas ordenadas
        familias_listas = {
            familia: sorted(list(configs))
            for familia, configs in familias.items()
        }

        return familias_listas

    @staticmethod
    def _extraer_familia_de_codigo(codigo_config: str) -> str:
        """
        Extrae el nombre de la familia del modelo desde el código de configuración.

        Ejemplos reales de tu índice:
        - "mask2former_large_ade_baja_sensibilidad" -> "mask2former"
        - "bodypix_mobilenet_050_estandar" -> "bodypix"
        - "sam2_hiera_large_auto_generacion" -> "sam2"
        - "sam2_hiera_large_box_prompts" -> "sam2_prompts"
        - "yolov8_nano_estandar" -> "yolov8"
        - "oneformer_dinat_large_semantic" -> "oneformer"

        Args:
            codigo_config: Código de configuración completo

        Returns:
            Nombre de la familia del modelo
        """
        # Casos especiales: SAM2 con prompts
        if codigo_config.startswith('sam2'):
            # Si contiene palabras clave de prompts, es sam2_prompts
            if any(palabra in codigo_config for palabra in ['box', 'point', 'combined', 'prompts']):
                return 'sam2_prompts'
            else:
                # Es SAM2 auto (sin prompts)
                return 'sam2'

        # Caso general: primera palabra antes del primer guión bajo
        return codigo_config.split('_')[0]

    def mostrar_menu(self) -> int:
        """
        Muestra menú interactivo de selección.

        Returns:
            Índice de opción seleccionada, o -1 si cancela
        """
        print("\n" + "="*80)
        print("SELECCIÓN DE MODELOS PARA PROCESAMIENTO")
        print("="*80)

        if not self.modelos_por_familia:
            print("\n¡ERROR! No se encontraron configuraciones en el índice maestro")
            return -1

        # Orden preferido de familias
        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias que no estén en el orden preferido (por si acaso)
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        print("\nModelos disponibles:\n")

        for idx, familia in enumerate(familias_disponibles, start=1):
            num_configs = len(self.modelos_por_familia[familia])
            nombre_display = familia.upper().replace('_', ' ')
            print(f"{idx}. {nombre_display}")
            print(f"   Configuraciones: {num_configs}\n")

        print(f"{len(familias_disponibles) + 1}. TODOS LOS MODELOS")
        total_configs = sum(len(configs) for configs in self.modelos_por_familia.values())
        print(f"   Total: {total_configs} configuraciones\n")

        print(f"{len(familias_disponibles) + 2}. SELECCIÓN MÚLTIPLE (separar con comas)")

        print("\n" + "="*80)

        while True:
            try:
                seleccion = input(f"\nSeleccione opción [1-{len(familias_disponibles) + 2}]: ").strip()

                if not seleccion:
                    print("Selección vacía. Cancelando...")
                    return -1

                opcion = int(seleccion)

                if 1 <= opcion <= len(familias_disponibles) + 2:
                    return opcion
                else:
                    print(f"Por favor ingrese un número entre 1 y {len(familias_disponibles) + 2}")

            except ValueError:
                print("Entrada inválida. Por favor ingrese un número")

            except KeyboardInterrupt:
                print("\n\nSelección cancelada por el usuario")
                return -1

    def seleccion_multiple(self) -> List[str]:
        """
        Permite selección múltiple de familias.

        Returns:
            Lista de familias seleccionadas
        """
        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias adicionales no ordenadas
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        print("\n" + "="*80)
        print("SELECCIÓN MÚLTIPLE")
        print("="*80)
        print("\nIngrese los números separados por comas (ej: 1,3,5)\n")

        for idx, familia in enumerate(familias_disponibles, start=1):
            nombre_display = familia.upper().replace('_', ' ')
            num_configs = len(self.modelos_por_familia[familia])
            print(f"{idx}. {nombre_display} ({num_configs} configs)")

        seleccion = input("\nSelección: ").strip()

        if not seleccion:
            return []

        try:
            indices = [int(x.strip()) for x in seleccion.split(',')]
            familias_sel = []

            for idx in indices:
                if 1 <= idx <= len(familias_disponibles):
                    familias_sel.append(familias_disponibles[idx - 1])
                else:
                    print(f"Advertencia: índice {idx} fuera de rango, ignorado")

            return familias_sel

        except ValueError:
            print("Error: formato inválido")
            return []

    def expandir_seleccion(self, familias_seleccionadas: List[str]) -> List[str]:
        """
        Expande familias seleccionadas a lista completa de configuraciones.

        Args:
            familias_seleccionadas: Lista de códigos de familia

        Returns:
            Lista completa de códigos de configuración
        """
        if 'TODAS' in familias_seleccionadas:
            configs = []
            for familia_configs in self.modelos_por_familia.values():
                configs.extend(familia_configs)
            return sorted(configs)

        configs = []
        for familia in familias_seleccionadas:
            if familia in self.modelos_por_familia:
                configs.extend(self.modelos_por_familia[familia])

        return sorted(configs)

    def confirmar_seleccion(self, configuraciones: List[str]) -> bool:
        """
        Muestra resumen y solicita confirmación.

        Args:
            configuraciones: Lista de configuraciones seleccionadas

        Returns:
            True si usuario confirma, False si cancela
        """
        print("\n" + "="*80)
        print("RESUMEN DE SELECCIÓN")
        print("="*80)
        print(f"\nTotal de configuraciones seleccionadas: {len(configuraciones)}")

        if len(configuraciones) <= 20:
            print("\nConfiguraciones:")
            for config in configuraciones:
                print(f"  - {config}")
        else:
            print("\nPrimeras 10 configuraciones:")
            for config in configuraciones[:10]:
                print(f"  - {config}")
            print(f"  ... y {len(configuraciones) - 10} más")

        print("\n" + "="*80)

        while True:
            confirmacion = input("\n¿Continuar con esta selección? [S/n]: ").strip().lower()

            if confirmacion in ['s', 'si', 'yes', '']:
                return True
            elif confirmacion in ['n', 'no']:
                return False
            else:
                print("Por favor responda 's' o 'n'")

    def seleccionar(self) -> Optional[List[str]]:
        """
        Proceso completo de selección interactiva.

        Returns:
            Lista de configuraciones seleccionadas, o None si se cancela
        """
        opcion = self.mostrar_menu()

        if opcion == -1:
            return None

        familias_ordenadas = ['bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts', 'yolov8']
        familias_disponibles = [f for f in familias_ordenadas if f in self.modelos_por_familia]

        # Agregar familias adicionales
        for familia in sorted(self.modelos_por_familia.keys()):
            if familia not in familias_disponibles:
                familias_disponibles.append(familia)

        if opcion == len(familias_disponibles) + 1:
            # TODAS
            familias_sel = ['TODAS']
        elif opcion == len(familias_disponibles) + 2:
            # SELECCIÓN MÚLTIPLE
            familias_sel = self.seleccion_multiple()
            if not familias_sel:
                print("\nSelección cancelada")
                return None
        else:
            # FAMILIA INDIVIDUAL
            familias_sel = [familias_disponibles[opcion - 1]]

        configuraciones = self.expandir_seleccion(familias_sel)

        if not configuraciones:
            print("\nError: No se seleccionaron configuraciones válidas")
            return None

        if self.confirmar_seleccion(configuraciones):
            return configuraciones
        else:
            print("\nSelección cancelada")
            return None

In [39]:
# =============================================================================
# ORQUESTADOR PRINCIPAL DE FASE 2A
# =============================================================================

class OrquestadorFase2A:
    """
    Orquestador principal para ejecución de Fase 2A.

    Coordina el pipeline completo de evaluación cuantitativa:
    - Gestión de checkpoint para procesamiento resumible
    - Carga de ground truth y máscaras predichas
    - Cálculo de métricas clásicas, geométricas y texturales
    - Consolidación de resultados en múltiples formatos
    - Generación de estadísticas y rankings

    Attributes:
        config: Configuración global de Fase 2A
        configuraciones_seleccionadas: Lista de códigos de configuración a procesar
        logger: Logger configurado según nivel de verbosidad
        checkpoint: Sistema de checkpoint para reanudar procesamiento
        resultados_consolidados: Lista acumulativa de resultados para CSV
        estadisticas_omisiones: Registro de configuraciones omitidas por foto
        indice_maestro: Índice maestro cargado desde Fase 1
        fotografias_disponibles: Lista de estructuras con información de fotografías

    Notes:
        - Las configuraciones con ruta_mascara=null se omiten automáticamente
        - El checkpoint se actualiza después de cada fotografía procesada
        - Los errores en cálculo de métricas se registran como NaN sin detener ejecución
    """

    def __init__(self, config: ConfiguracionFase2A, configuraciones: List[str]):
        """
        Inicializa el orquestador con configuración y lista de configuraciones.

        Args:
            config: Configuración global del sistema
            configuraciones: Lista de códigos de configuración seleccionados por el usuario

        Raises:
            FileNotFoundError: Si no existe el índice maestro de Fase 1
        """
        self.config = config
        self.configuraciones_seleccionadas = configuraciones
        self.logger = configurar_logging(config.nivel_logging, "OrquestadorFase2A")

        config.crear_directorios()

        self.checkpoint: Optional[CheckpointFase2A] = None
        self.resultados_consolidados: List[Dict] = []
        self.estadisticas_omisiones: Dict[str, List[str]] = {}

        # Cargar índice maestro de Fase 1
        ruta_indice = config.ruta_fase1 / "indice_maestro.json"

        if not ruta_indice.exists():
            raise FileNotFoundError(f"Índice maestro no encontrado: {ruta_indice}")

        with open(ruta_indice, 'r', encoding='utf-8') as f:
            self.indice_maestro = json.load(f)

        # Extraer información de fotografías disponibles
        self.fotografias_disponibles = []

        for foto_id in sorted(self.indice_maestro.keys()):
            datos_foto = self.indice_maestro[foto_id]

            foto_info = {
                "codigo_foto": foto_id,
                "ruta_imagen": datos_foto["rutas"]["imagen"],
                "ruta_ground_truth": datos_foto["rutas"]["ground_truth"],
                "ruta_caracteristicas": datos_foto["rutas"]["caracteristicas"],
                "modelos_disponibles": datos_foto["modelos_disponibles"]
            }

            self.fotografias_disponibles.append(foto_info)

        self.logger.info(f"Índice maestro cargado: {len(self.fotografias_disponibles)} fotografías")

    def cargar_checkpoint(self) -> None:
        """
        Carga checkpoint existente si está habilitado.

        Permite reanudar procesamiento desde fotografía interrumpida.
        Actualiza la lista de fotografías pendientes basándose en fotos ya procesadas.
        """
        ruta_checkpoint = self.config.ruta_salida / "checkpoint_fase2a.json"

        if self.config.usar_checkpoint and ruta_checkpoint.exists():
            try:
                with open(ruta_checkpoint, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    self.checkpoint = CheckpointFase2A.from_dict(data)

                self.logger.info(f"Checkpoint cargado: {len(self.checkpoint.fotos_procesadas)} fotos ya procesadas")

            except Exception as e:
                self.logger.warning(f"Error cargando checkpoint: {e}. Iniciando desde cero")
                self.checkpoint = CheckpointFase2A()
        else:
            self.checkpoint = CheckpointFase2A()

        # Actualizar fotos pendientes
        codigos_fotos = [f["codigo_foto"] for f in self.fotografias_disponibles]

        self.checkpoint.fotos_pendientes = [
            codigo for codigo in codigos_fotos
            if codigo not in self.checkpoint.fotos_procesadas
        ]

        self.checkpoint.configuraciones_totales = len(self.configuraciones_seleccionadas)

    def guardar_checkpoint(self) -> None:
        """
        Guarda estado actual del checkpoint.

        Se ejecuta después de cada fotografía procesada para permitir
        reanudación en caso de interrupción.
        """
        if not self.config.usar_checkpoint:
            return

        self.checkpoint.ultima_actualizacion = datetime.now().isoformat()

        ruta_checkpoint = self.config.ruta_salida / "checkpoint_fase2a.json"

        guardar_json_seguro(self.checkpoint.to_dict(), ruta_checkpoint, self.logger)

    def procesar_fotografia(self, info_foto: Dict) -> None:
        """
        Procesa una fotografía completa con todas sus configuraciones seleccionadas.

        Pipeline de procesamiento:
        1. Carga ground truth
        2. Convierte GT a geometría Shapely
        3. Calcula métricas base del GT
        4. Para cada configuración seleccionada disponible:
           - Verifica existencia de máscara NPZ
           - Carga máscara predicha
           - Calcula métricas clásicas (IoU, Dice, Precision, Recall, F1)
           - Calcula métricas geométricas Shapely
           - Calcula métricas comparativas
           - Calcula métricas texturales Mahotas
        5. Guarda resultados individuales en JSON
        6. Actualiza checkpoint

        Args:
            info_foto: Diccionario con información de la fotografía:
                - codigo_foto: Identificador único
                - ruta_imagen: Path a imagen original
                - ruta_ground_truth: Path a máscara GT
                - ruta_caracteristicas: Path a JSON de características
                - modelos_disponibles: Dict de configuraciones disponibles

        Notes:
            - Configuraciones sin máscara NPZ (ruta_mascara=null) se omiten
            - Errores en conversión a Shapely se registran como omisiones
            - Mahotas se omite si no existe imagen original RGB
        """
        codigo_foto = info_foto["codigo_foto"]

        if self.config.nivel_logging in [NivelLogging.NORMAL, NivelLogging.DETAILED, NivelLogging.DEBUG]:
            self.logger.info(f"\n{'='*80}")
            self.logger.info(f"Procesando fotografía: {codigo_foto}")
            self.logger.info(f"{'='*80}")

        # Cargar ground truth
        ruta_gt = Path(info_foto["ruta_ground_truth"])

        if not ruta_gt.exists():
            self.logger.error(f"Ground truth no encontrado para {codigo_foto}. Omitiendo fotografía")
            return

        mascara_gt = cargar_mascara_npz(ruta_gt, logger=self.logger)

        if mascara_gt is None:
            self.logger.error(f"Error cargando ground truth de {codigo_foto}. Omitiendo fotografía")
            return

        # Convertir GT a Shapely
        geom_gt = mascara_a_shapely(mascara_gt, self.logger)

        if geom_gt is None:
            self.logger.error(f"Error convirtiendo GT a Shapely para {codigo_foto}. Omitiendo fotografía")
            return

        # Dimensiones de la imagen
        dimensiones_imagen = (mascara_gt.shape[0], mascara_gt.shape[1])

        # Métricas del ground truth
        metricas_gt = calcular_metricas_shapely(geom_gt, dimensiones_imagen, self.logger)

        # Cargar imagen original para Mahotas
        ruta_imagen_original = Path(info_foto["ruta_imagen"])

        if ruta_imagen_original.exists():
            imagen_rgb = np.array(Image.open(ruta_imagen_original).convert("RGB"))
        else:
            self.logger.warning(f"Imagen original no encontrada: {ruta_imagen_original}. Mahotas omitido")
            imagen_rgb = None

        # Estructura de resultados
        resultados_foto = {
            "codigo_foto": codigo_foto,
            "dimensiones_imagen": {
                "altura": dimensiones_imagen[0],
                "ancho": dimensiones_imagen[1]
            },
            "ground_truth_metricas": metricas_gt,
            "configuraciones": []
        }

        configs_procesadas = 0
        configs_omitidas = 0

        # Iterar sobre configuraciones seleccionadas que existen en esta foto
        modelos_disponibles_foto = info_foto["modelos_disponibles"]

        for config_codigo in self.configuraciones_seleccionadas:
            # Verificar si esta configuración existe para esta foto
            if config_codigo not in modelos_disponibles_foto:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"  Config {config_codigo} no disponible para {codigo_foto}")
                configs_omitidas += 1

                # Registrar omisión
                modelo = config_codigo.split('_')[0]
                clave_omision = f"{codigo_foto}_{config_codigo}"
                if modelo not in self.estadisticas_omisiones:
                    self.estadisticas_omisiones[modelo] = []
                self.estadisticas_omisiones[modelo].append(clave_omision)
                continue

            if self.config.nivel_logging == NivelLogging.DEBUG:
                self.logger.debug(f"  Procesando config: {config_codigo}")

            # Obtener información de la configuración
            config_info = modelos_disponibles_foto[config_codigo]

            # Verificar si tiene máscara NPZ (puede ser null)
            if config_info.get("ruta_mascara") is None:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"    Config {config_codigo} no tiene máscara NPZ (solo metadata)")
                configs_omitidas += 1

                # Registrar omisión
                modelo = config_codigo.split('_')[0]
                clave_omision = f"{codigo_foto}_{config_codigo}"
                if modelo not in self.estadisticas_omisiones:
                    self.estadisticas_omisiones[modelo] = []
                self.estadisticas_omisiones[modelo].append(clave_omision)
                continue

            ruta_mascara = Path(config_info["ruta_mascara"])

            if not ruta_mascara.exists():
                self.logger.warning(f"    Máscara no encontrada: {ruta_mascara}")
                configs_omitidas += 1
                continue

            # Cargar máscara predicha
            mascara_pred = cargar_mascara_npz(ruta_mascara, logger=self.logger)

            if mascara_pred is None:
                self.logger.warning(f"    Error cargando máscara {config_codigo} para {codigo_foto}")
                configs_omitidas += 1
                continue

            # Calcular métricas clásicas
            metricas_clasicas = calcular_metricas_clasicas(mascara_pred, mascara_gt)

            # Convertir predicción a Shapely
            geom_pred = mascara_a_shapely(mascara_pred, self.logger)

            if geom_pred is None:
                self.logger.warning(f"    Error convirtiendo predicción a Shapely: {config_codigo}")
                configs_omitidas += 1
                continue

            # Métricas Shapely de predicción
            metricas_shapely_pred = calcular_metricas_shapely(geom_pred, dimensiones_imagen, self.logger)

            # Métricas comparativas
            metricas_comparativas = calcular_metricas_comparativas(
                geom_pred, geom_gt, mascara_pred, mascara_gt, self.logger
            )

            # Métricas Mahotas
            if imagen_rgb is not None:
                metricas_mahotas = calcular_metricas_mahotas(
                    imagen_rgb, mascara_pred,
                    ancho_borde=self.config.ancho_borde_px,
                    max_dimension=self.config.max_dimension_mahotas,
                    logger=self.logger
                )
            else:
                metricas_mahotas = {}

            # Consolidar resultados de configuración
            resultado_config = {
                "config_codigo": config_codigo,
                "modelo": config_codigo.split('_')[0],
                "metricas_clasicas": metricas_clasicas,
                "metricas_shapely": metricas_shapely_pred,
                "metricas_comparativas": metricas_comparativas,
                "metricas_mahotas": metricas_mahotas
            }

            resultados_foto["configuraciones"].append(resultado_config)

            # Agregar a consolidado para CSV
            fila_csv = {
                "codigo_foto": codigo_foto,
                "config_codigo": config_codigo,
                "modelo": config_codigo.split('_')[0],
                **metricas_clasicas,
                **metricas_shapely_pred,
                **metricas_comparativas,
                **metricas_mahotas
            }

            self.resultados_consolidados.append(fila_csv)

            configs_procesadas += 1

        # Guardar JSON individual
        if self.config.guardar_json_individuales:
            ruta_json_individual = self.config.ruta_salida / "metricas_individuales" / f"{codigo_foto}_metricas_completas.json"
            guardar_json_seguro(resultados_foto, ruta_json_individual, self.logger)

        # Actualizar checkpoint
        self.checkpoint.fotos_procesadas.add(codigo_foto)

        if codigo_foto in self.checkpoint.fotos_pendientes:
            self.checkpoint.fotos_pendientes.remove(codigo_foto)

        self.guardar_checkpoint()

        if self.config.nivel_logging in [NivelLogging.NORMAL, NivelLogging.DETAILED]:
            self.logger.info(f"  Configuraciones procesadas: {configs_procesadas}")
            self.logger.info(f"  Configuraciones omitidas: {configs_omitidas}")

    def ejecutar(self) -> None:
        """
        Ejecuta el pipeline completo de Fase 2A.

        Procesa todas las fotografías pendientes y genera archivos de salida:
        - JSONs individuales por fotografía (opcional)
        - CSV consolidado con todas las métricas
        - Estadísticas por modelo
        - Ranking TOP-10 de configuraciones
        - Reporte de configuraciones omitidas

        El procesamiento es resumible mediante checkpoint si está habilitado.
        """
        self.logger.info("\nIniciando Fase 2A - Evaluación Cuantitativa")
        self.logger.info(f"Configuraciones a evaluar: {len(self.configuraciones_seleccionadas)}")
        self.logger.info(f"Fotografías disponibles: {len(self.fotografias_disponibles)}")

        self.cargar_checkpoint()

        self.logger.info(f"Fotografías pendientes: {len(self.checkpoint.fotos_pendientes)}")
        self.logger.info(f"Fotografías ya procesadas: {len(self.checkpoint.fotos_procesadas)}\n")

        # Procesar cada fotografía
        for info_foto in self.fotografias_disponibles:
            codigo_foto = info_foto["codigo_foto"]

            if codigo_foto in self.checkpoint.fotos_procesadas:
                if self.config.nivel_logging == NivelLogging.DEBUG:
                    self.logger.debug(f"Omitiendo {codigo_foto} (ya procesada)")
                continue

            try:
                self.procesar_fotografia(info_foto)

            except Exception as e:
                self.logger.error(f"Error procesando {codigo_foto}: {e}")
                import traceback
                traceback.print_exc()
                continue

        self.logger.info("\nProcesamiento de fotografías completado")

        # Generar archivos consolidados
        if self.config.guardar_csv_consolidado:
            self.generar_csv_consolidado()

        if self.config.guardar_estadisticas:
            self.generar_estadisticas_por_modelo()

        self.generar_ranking_top10()

        self.guardar_reporte_omisiones()

        self.logger.info("\nFase 2A completada exitosamente")

    def generar_csv_consolidado(self) -> None:
        """
        Genera CSV consolidado con todas las métricas.

        Formato: Una fila por combinación foto-configuración.
        Columnas: código_foto, config_codigo, modelo + ~60 métricas

        Output: metricas_agregadas/todas_metricas.csv
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos para consolidar en CSV")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            ruta_csv = self.config.ruta_salida / "metricas_agregadas" / "todas_metricas.csv"

            df.to_csv(ruta_csv, index=False, encoding='utf-8')

            self.logger.info(f"CSV consolidado guardado: {ruta_csv}")
            self.logger.info(f"  Filas: {len(df)}")
            self.logger.info(f"  Columnas: {len(df.columns)}")

        except Exception as e:
            self.logger.error(f"Error generando CSV consolidado: {e}")

    def generar_ranking_top10(self) -> None:
        """
        Genera ranking TOP-10 de configuraciones ordenadas por IoU medio.

        Para cada configuración calcula:
        - IoU medio, desviación estándar, mínimo, máximo
        - Número de fotografías evaluadas
        - Modelo al que pertenece

        Output: metricas_agregadas/ranking_configuraciones.csv
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos para generar ranking")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            # Agrupar por configuración y calcular estadísticas de IoU
            ranking = df.groupby('config_codigo').agg({
                'iou': ['mean', 'std', 'min', 'max', 'count'],
                'modelo': 'first'
            }).reset_index()

            ranking.columns = ['config_codigo', 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'num_fotos', 'modelo']

            # Ordenar por IoU medio descendente
            ranking = ranking.sort_values('iou_mean', ascending=False)

            # TOP-10
            top10 = ranking.head(10)

            ruta_ranking = self.config.ruta_salida / "metricas_agregadas" / "ranking_configuraciones.csv"

            top10.to_csv(ruta_ranking, index=False, encoding='utf-8')

            self.logger.info(f"Ranking TOP-10 guardado: {ruta_ranking}")

        except Exception as e:
            self.logger.error(f"Error generando ranking: {e}")

    def generar_estadisticas_por_modelo(self) -> None:
        """
        Genera estadísticas agregadas por modelo.

        Para cada modelo calcula:
        - Número de configuraciones únicas
        - Número total de evaluaciones
        - Estadísticas de IoU: media, desviación, min, max, mediana

        Output: metricas_agregadas/estadisticas_por_modelo.json
        """
        try:
            if len(self.resultados_consolidados) == 0:
                self.logger.warning("No hay datos suficientes para generar estadísticas")
                return

            df = pd.DataFrame(self.resultados_consolidados)

            ruta_stats = self.config.ruta_salida / "metricas_agregadas" / "estadisticas_por_modelo.json"

            stats_por_modelo = {}

            for modelo in df['modelo'].unique():
                df_modelo = df[df['modelo'] == modelo]

                stats_por_modelo[modelo] = {
                    'num_configuraciones': int(df_modelo['config_codigo'].nunique()),
                    'num_evaluaciones': int(len(df_modelo)),
                    'iou': {
                        'mean': float(df_modelo['iou'].mean()),
                        'std': float(df_modelo['iou'].std()),
                        'min': float(df_modelo['iou'].min()),
                        'max': float(df_modelo['iou'].max()),
                        'median': float(df_modelo['iou'].median())
                    }
                }

            guardar_json_seguro(stats_por_modelo, ruta_stats, self.logger)

            self.logger.info(f"Estadísticas por modelo guardadas: {ruta_stats}")

        except Exception as e:
            self.logger.error(f"Error generando estadísticas: {e}")

    def guardar_reporte_omisiones(self) -> None:
        """
        Guarda reporte detallado de configuraciones omitidas.

        Útil para debugging y auditoría del proceso de evaluación.
        Incluye:
        - Total de omisiones
        - Omisiones por modelo
        - Lista detallada de combinaciones foto-config omitidas

        Output: metricas_agregadas/configuraciones_omitidas.json
        """
        try:
            if len(self.estadisticas_omisiones) == 0:
                self.logger.info("No hay omisiones para reportar")
                return

            ruta_omisiones = self.config.ruta_salida / "metricas_agregadas" / "configuraciones_omitidas.json"

            reporte = {
                "total_omisiones": sum(len(v) for v in self.estadisticas_omisiones.values()),
                "por_modelo": {
                    modelo: {
                        "count": len(omisiones),
                        "combinaciones": omisiones
                    }
                    for modelo, omisiones in self.estadisticas_omisiones.items()
                }
            }

            guardar_json_seguro(reporte, ruta_omisiones, self.logger)

            self.logger.info(f"Reporte de omisiones guardado: {ruta_omisiones}")

        except Exception as e:
            self.logger.error(f"Error guardando reporte de omisiones: {e}")

In [37]:
# =============================================================================
# FUNCIÓN PRINCIPAL DE EJECUCIÓN INTERACTIVA
# =============================================================================

def ejecutar_fase2a_interactivo(ruta_base_tfm: str = "/content/drive/MyDrive/TFM") -> Optional[OrquestadorFase2A]:
    """
    Ejecuta el pipeline completo de Fase 2A con selección interactiva.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM

    Returns:
        OrquestadorFase2A con resultados, o None si cancelado
    """
    print("\n" + "="*80)
    print("FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO")
    print("Trabajo Fin de Máster - Universidad Oberta de Catalunya")
    print("="*80 + "\n")

    # Configuración
    config = ConfiguracionFase2A(
        ruta_base_tfm=Path(ruta_base_tfm),
        max_dimension_mahotas=1024,
        ancho_borde_px=5,
        usar_checkpoint=True,
        guardar_json_individuales=True,
        guardar_csv_consolidado=True,
        guardar_estadisticas=True,
        nivel_logging=NivelLogging.NORMAL
    )

    # Cargar índice maestro
    ruta_indice = config.ruta_fase1 / "indice_maestro.json"

    if not ruta_indice.exists():
        raise FileNotFoundError(f"Índice maestro no encontrado: {ruta_indice}")

    with open(ruta_indice, 'r', encoding='utf-8') as f:
        indice_maestro = json.load(f)

    # Selección de modelos
    logger_temp = logging.getLogger('Selector')
    logger_temp.setLevel(logging.INFO)

    selector = SelectorModelos(indice_maestro, logger_temp)

    configuraciones_seleccionadas = selector.seleccionar()

    if configuraciones_seleccionadas is None:
        print("\nEjecución cancelada por el usuario")
        return None

    # Configuración de nivel de logging
    print("\n" + "="*80)
    print("CONFIGURACIÓN DE NIVEL DE LOGGING")
    print("="*80)
    print("\n1. MINIMAL - Solo resumen por foto")
    print("2. NORMAL - Progreso con detalle moderado (recomendado)")
    print("3. DETAILED - Información detallada por configuración")
    print("4. DEBUG - Máximo detalle para debugging")

    while True:
        try:
            nivel_input = input("\nSeleccione nivel [1-4, default=2]: ").strip()

            if nivel_input == '' or nivel_input == '2':
                config.nivel_logging = NivelLogging.NORMAL
                break
            elif nivel_input == '1':
                config.nivel_logging = NivelLogging.MINIMAL
                break
            elif nivel_input == '3':
                config.nivel_logging = NivelLogging.DETAILED
                break
            elif nivel_input == '4':
                config.nivel_logging = NivelLogging.DEBUG
                break
            else:
                print("Por favor ingrese un número entre 1 y 4")

        except KeyboardInterrupt:
            print("\n\nEjecución cancelada por el usuario")
            return None

    # Resumen de configuración
    print(f"\nConfiguración:")
    print(f"  - Ruta base: {config.ruta_base_tfm}")
    print(f"  - Ruta Fase 1: {config.ruta_fase1}")
    print(f"  - Ruta salida: {config.ruta_salida}")
    print(f"  - Configuraciones a procesar: {len(configuraciones_seleccionadas)}")
    print(f"  - Nivel logging: {config.nivel_logging.value}")
    print(f"  - Checkpoint: {config.usar_checkpoint}")
    print()

    # Crear orquestador y ejecutar
    orquestador = OrquestadorFase2A(config, configuraciones_seleccionadas)

    orquestador.ejecutar()

    return orquestador

In [44]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":

    RUTA_BASE_TFM = "/content/drive/MyDrive/TFM"

    try:
        orquestador = ejecutar_fase2a_interactivo(RUTA_BASE_TFM)

        if orquestador is None:
            print("\nEjecución cancelada")
            sys.exit(0)

        # Resumen final
        print("\n" + "="*80)
        print("RESUMEN DE EJECUCIÓN")
        print("="*80)
        print(f"Fotografías procesadas: {len(orquestador.checkpoint.fotos_procesadas) if orquestador.checkpoint else 'N/A'}")
        print(f"Evaluaciones exitosas: {len(orquestador.resultados_consolidados)}")

        if orquestador.estadisticas_omisiones:
            total_omisiones = sum(len(v) for v in orquestador.estadisticas_omisiones.values())
            print(f"Configuraciones omitidas: {total_omisiones}")

        print(f"\nResultados disponibles en:")
        print(f"  - JSONs individuales: {orquestador.config.ruta_salida / 'metricas_individuales'}")
        print(f"  - CSV consolidado: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'todas_metricas.csv'}")
        print(f"  - Ranking TOP-10: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'ranking_configuraciones.csv'}")
        print(f"  - Estadísticas: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'estadisticas_por_modelo.json'}")
        print(f"  - Reporte omisiones: {orquestador.config.ruta_salida / 'metricas_agregadas' / 'configuraciones_omitidas.json'}")

        print("\n" + "="*80)
        print("FASE 2A COMPLETADA EXITOSAMENTE")
        print("="*80)

    except KeyboardInterrupt:
        print("\n\nEjecución interrumpida por el usuario")
        sys.exit(1)

    except Exception as e:
        print(f"\nERROR DURANTE EJECUCIÓN: {e}")
        import traceback
        traceback.print_exc()
        raise



FASE 2A - EVALUACIÓN CUANTITATIVA Y ANÁLISIS GEOMÉTRICO
Trabajo Fin de Máster - Universidad Oberta de Catalunya


SELECCIÓN DE MODELOS PARA PROCESAMIENTO

Modelos disponibles:

1. BODYPIX
   Configuraciones: 24

2. MASK2FORMER
   Configuraciones: 19

3. ONEFORMER
   Configuraciones: 36

4. SAM2
   Configuraciones: 12

5. SAM2 PROMPTS
   Configuraciones: 32

6. YOLOV8
   Configuraciones: 20

7. TODOS LOS MODELOS
   Total: 143 configuraciones

8. SELECCIÓN MÚLTIPLE (separar con comas)


Seleccione opción [1-8]: 1

RESUMEN DE SELECCIÓN

Total de configuraciones seleccionadas: 24

Primeras 10 configuraciones:
  - bodypix_mobilenet_v1_050_baja_sensibilidad_t0_3
  - bodypix_mobilenet_v1_050_baja_sensibilidad_t0_4
  - bodypix_mobilenet_v1_050_baja_sensibilidad_t0_5
  - bodypix_mobilenet_v1_050_sensibilidad_alta_t0_15
  - bodypix_mobilenet_v1_050_sensibilidad_alta_t0_2
  - bodypix_mobilenet_v1_050_sensibilidad_alta_t0_25
  - bodypix_mobilenet_v1_050_sensibilidad_media_t0_2
  - bodypix_mobilen

INFO:OrquestadorFase2A:Índice maestro cargado: 20 fotografías


[20:51:55] INFO     | 
Iniciando Fase 2A - Evaluación Cuantitativa


INFO:OrquestadorFase2A:
Iniciando Fase 2A - Evaluación Cuantitativa


[20:51:55] INFO     | Configuraciones a evaluar: 24


...

DEBUG:OrquestadorFase2A:Claves en NPZ: ['person_mask', 'probability_mask', 'threshold_value', 'body_parts_mask', 'grupo_cara', 'grupo_torso', 'grupo_brazos', 'grupo_manos', 'grupo_piernas', 'grupo_pies']


[20:52:06] DEBUG    | Cargada máscara unificada, shape: (4096, 2734)


DEBUG:OrquestadorFase2A:Cargada máscara unificada, shape: (4096, 2734)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.





Ejecución interrumpida por el usuario
Traceback (most recent call last):
  File "/tmp/ipython-input-2135785638.py", line 10, in <cell line: 0>
    orquestador = ejecutar_fase2a_interactivo(RUTA_BASE_TFM)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-4285730203.py", line 98, in ejecutar_fase2a_interactivo
    orquestador.ejecutar()
  File "/tmp/ipython-input-2093944007.py", line 375, in ejecutar
    self.procesar_fotografia(info_foto)
  File "/tmp/ipython-input-2093944007.py", line 273, in procesar_fotografia
    geom_pred = mascara_a_shapely(mascara_pred, self.logger)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1528737909.py", line 209, in mascara_a_shapely
    contornos = measure.find_contours(mascara_binaria, level=0.5)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/skimage/measure/_find_contours.py", line 149, in find_contours
   

TypeError: object of type 'NoneType' has no len()